In [ ]:
import yaml
import os
import logging
import plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from evaluation.evaluation_lib import aggregate_all_results

log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
if not log.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)
    ch.setFormatter(logging.Formatter('%(levelname)s - %(message)s'))
    log.addHandler(ch)

In [ ]:
# Parse experiment yaml file
experiments_path="../experiments/full_euroc.yaml"

# Get experiment information from yaml file.
experiment_params = yaml.load(open(experiments_path))

# Get directory where all results are stored
results_dir = os.path.expandvars(experiment_params['results_dir'])

# Collect results
stats = aggregate_all_results(results_dir)

In [ ]:
# Store stats in a tidy Pandas DataFrame # TODO(Toni): this should be done in the evaluation_lib.py script...
def listify_stats(stats):
    """ Makes a list of lists out of the stats (for easy conversion into pandas dataframe) """
    stats_list = []
    for dataset_name in stats:
        for pipeline in stats[dataset_name]:
            result = stats[dataset_name][pipeline]
            if result != False:
                result = result['absolute_errors'].np_arrays['error_array']
                stats_list.append([dataset_name, pipeline, result])
    return stats_list

# Create or load Pandas DataFrame
df = pd.DataFrame()
log.info("Creating dataframe stats.")
df = pd.DataFrame.from_records(listify_stats(stats))
df.columns = ['Dataset Name', 'Pipe Type', 'ATE errors']
#df.set_index(['Dataset Name'], inplace = True)
    
    # Save dataframe as pickle for future use
    #df.to_pickle(all_stats_pickle_dir)

# Print df
df

In [ ]:
def boxplot(df):
    tidy = df.set_index(['Dataset Name'])
    tidy = tidy['ATE errors'].apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('ATE errors')
    tidy.reset_index(level=['Dataset Name'], drop=False, inplace=True)
    tidy.sort_values('Dataset Name', inplace=True)
    fig = px.box(tidy, x='Dataset Name', y="ATE errors", points="all")

    fig.update_layout(
    title=go.layout.Title(
        text="ATE in EuRoC dataset"
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='Datasets'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="ATE [m]"
            ),
        rangemode='tozero'
        ),
    template='plotly_white'
    )
    return fig

In [ ]:
# Generate figures
figure = boxplot(df)

In [ ]:
figure.show()